# Estimation with SPDE

This document aims at demonstrating the use of SPDE for performing Estimation. It is based on Scotland Data

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
return false;
}

In [ ]:
import gstlearn as gl
import matplotlib.pyplot as plt
import gstlearn.plot as gp
import os 
import numpy as np

Getting the Data Bases from the repository.

In [ ]:
!wget -q https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Temperatures.NF
!wget -q https://soft.minesparis.psl.eu/gstlearn/data/Scotland/Scotland_Elevations.NF

The input Data Base (called **temperatures**) contains the target variable (**January_temp**). Note that this data base also contains the elevation variable (called **Elevation**) which is assigned the locator **f** (for external drift). We finally add a selection (called **sel**) which only consider the samples where the temperature is actually calculated.

In [ ]:
filename = "Scotland_Temperatures.NF"
temperatures = gl.Db.createFromNF(filename)
temperatures.setLocator("January_temp", gl.ELoc.Z)
temperatures.setLocator("Elevation",gl.ELoc.F)
iuid = temperatures.addSelection(np.invert(np.isnan(temperatures["J*"])),"sel")
temperatures.display()
ax = temperatures.plot("January_temp")
ax.decoration(title="Temperature data")

The output file is a grid (called **grid**). It contains an active selection (**inshore**) as well as the elevation over the field (called **Elevation**). Note that this variable is assigned the locator **f** (external drift).

In [ ]:
filename = "Scotland_Elevations.NF"
grid = gl.DbGrid.createFromNF(filename)
grid.display()
ax = grid.plot("Elevation")
ax.decoration(title="Elevation")

Calculate the omni-directional variogram of the temperature for 18 lags of 25.

In [ ]:
vparam = gl.VarioParam.createOmniDirection(npas=18, dpas=25)
vario = gl.Vario(vparam,temperatures)
vario.compute()
vario.display()
ax = gp.variogram(vario)
ax.decoration(title="Variogram of Temperature (Raw)")

We calculate the variogram (using the same calculation parameters) based on the residuals after a trend has been removed. This trend is considered as a linear combinaison of the external drift information.

In [ ]:
vparam = gl.VarioParam.createOmniDirection(npas=18, dpas=25)
vario = gl.Vario(vparam,temperatures)
md = gl.Model()
md.setDriftIRF(order=0, nfex=1)
vario.compute(model=md)
vario.display()
ax = gp.variogram(vario)

Fit the variogram of residuals in a model having drifts. Some constraints have been added during the fitting step.

In [ ]:
model = md

structs = [gl.ECov.NUGGET,gl.ECov.BESSEL_K]

consNug = gl.ConsItem.define(gl.EConsElem.SILL,0, type = gl.EConsType.UPPER,value = 0.1)

cons1P  = gl.ConsItem.define(gl.EConsElem.PARAM,1, type = gl.EConsType.EQUAL,value = 1)
cons1Rm = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.LOWER,value = 100)
cons1RM = gl.ConsItem.define(gl.EConsElem.RANGE,1, type = gl.EConsType.UPPER,value = 350)

a = gl.Constraints()
a.addItem(consNug)
a.addItem(cons1P)
a.addItem(cons1Rm)
a.addItem(cons1RM)

err = model.fit(vario,structs,constraints=a)
model.display()
ax = gp.varmod(vario,model)
ax.decoration(title="Vario of the residuals")

Derive the parameter of a Global Trend (composed of the Elevation as a drift function) using the SPDE formalism.

In [ ]:
spde = gl.SPDE(model,grid,temperatures,gl.ESPDECalcMode.KRIGING)
coeffs = spde.getCoeffs()
print("Trend coefficients:", coeffs)

Represent the scatter plot of the Temperature given the Elevation and add the Global Trend (calculated beforehand)

In [ ]:
ax = gp.correlation(temperatures, namex="Elevation", namey="*temp", asPoint=True)
if len(coeffs)>1:
    plt.plot([0,400], [coeffs[0],coeffs[0]+coeffs[1]*400])

We perform the Estimation in the SPDE framework (considering the Elevation as the Global Trend)

In [ ]:
err = gl.krigingSPDE(temperatures, grid, model)
ax = grid.plot(useSel=True)
ax.decoration(title="Temperature (using Elevation as global Trend)")

We also perform the Estimation by Kriging (using Elevation as External Drift). This estimation is performed in Unique Neighborhood.

In [ ]:
neighU = gl.NeighUnique.create();
gl.kriging(temperatures, grid, model, neighU);
ax = grid.plot(useSel=True)
ax.decoration(title="Temperature (with Elevation as External Drift)")

Comparing both estimates

In [ ]:
ax = gp.correlation(grid,namex="*estim",namey="*SPDE.*",asPoint=True, diagLine=True)
ax.decoration(xlabel="Kriging with External Drift", ylabel="SPDE Kriging")